In [1]:
from graphdatascience import GraphDataScience

In [2]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "neo4jneo4j")
DB = 'medical'

In [ ]:
gds = GraphDataScience(URI, AUTH, database=DB)
print(gds.version())

In [ ]:
graphname = "mygraph"
pipelinename = "mypipe"
modelname = "mymodel"

In [ ]:
RANDOM_SEED = 42

In [ ]:
if gds.graph.exists(graphname).exists:
    gds.graph.drop(gds.graph.get(graphname))
if gds.beta.pipeline.exists(pipelinename).exists:
    gds.pipeline.get(pipelinename).drop()
if gds.beta.model.exists(modelname).exists:
    gds.model.get(modelname).drop()

In [ ]:
G, result = gds.graph.project(graphname, 
  ['Patient','Diag','Drug','IHD'],
  {
    'IHD_RISK': {
      'orientation': 'UNDIRECTED'
    },
    'DIAGNOSED_WITH': {},
    'PRESCRIBED': {}
  }
);

In [ ]:
pipe, _ = gds.beta.pipeline.linkPrediction.create(pipelinename)

In [ ]:
pipe.addNodeProperty("beta.hashgnn",
    mutateProperty="embedding",
    iterations=4,
    heterogeneous=True,
    embeddingDensity=512,
    neighborInfluence=0.7,
    randomSeed=RANDOM_SEED,
    generateFeatures={'dimension': 1,  'densityLevel': 1},
    contextNodeLabels=G.node_labels(),
)

In [ ]:
pipe.addFeature('COSINE', nodeProperties = ['embedding'])

In [ ]:
pipe.configureSplit(trainFraction=0.5, testFraction=0.5, validationFolds=4)

In [ ]:
pipe.addLogisticRegression(tolerance=(0.1, 1.0), penalty=1.0, patience=5)

In [ ]:
pipe.addMLP(patience = 2, penalty = 1.0, hiddenLayerSizes = [4, 2])

In [ ]:
pipe.configureAutoTuning(maxTrials = 4)

In [ ]:
model, train_result = pipe.train(
    G,
    modelName = modelname,
    sourceNodeLabel = 'Patient',
    targetNodeLabel = "IHD",
    targetRelationshipType = 'IHD_RISK',
    metrics = ['AUCPR', 'OUT_OF_BAG_ERROR'],
    randomSeed = RANDOM_SEED
)

In [ ]:
predictions = model.predict_stream(G, concurrency=2, topN= 20, threshold= 0.1)

In [ ]:
predictions

In [ ]:
pipe.drop()
model.drop()
G.drop()

In [ ]:
gds.close()